| ![EEW logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/eew.jpg?raw=true) | ![EDGI logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/edgi.png?raw=true) |
|---|---|

#### This notebook is licensed under GPL 3.0. Please visit our Github repo for more information: https://github.com/edgi-govdata-archiving/ECHO-COVID19
#### The notebook was collaboratively authored by EDGI following our authorship protocol: https://docs.google.com/document/d/1CtDN5ZZ4Zv70fHiBTmWkDJ9mswEipX6eCYrwicP66Xw/
#### For more information about this project, visit https://www.environmentalenforcementwatch.org/

# Examining Data from Multiple EPA Programs

## Violations, Inspections and Enforcements for the Resource Conservation and Recovery Act (RCRA), Clean Water Act (CWA), Clean Air Act (CAA) and Combined Air Emissions

This notebook examines data from the EPA's Enforcement and Compliance History Online (ECHO) database (https://echo.epa.gov/). It includes information from EPA's programs covering air (the Clean Air Act, or CAA), water (the Clean Water Act, or CWA) and hazardous and other waste handling (the Resource Recovery and Conservation Act, or RCRA). ECHO data is available here on inspections, violations and enforcements by EPA, state and other agencies. Information made accessible here runs from the present day (the database is refreshed weekly) back to 2001, which is when the EPA believes the data to be most reliable. It is available at the Congressional District level for a selected state, and for counties and zip codes of your choosing. 

## How to Run
* A "cell" in a Jupyter notebook is a block of code performing a set of actions making available or using specific data.  The notebook works by running one cell after another, as the notebook user selects offered options.
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* **It is important to run cells in order because they depend on each other.**
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---
---

# **Let's begin!**

### Hover over the "[ ]" on the top left corner of the cell below and you should see a "play" button appear. Click on it to run the cell then move to the next one.

#### These first two cells give us access to some external Python code we will need.
### 1.  Bring in some code that is stored in a Github project.

In [ ]:
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git
!git clone https://github.com/edgi-govdata-archiving/ECHO-Cross-Program.git -b bugs_and_cleanup

### 2.  Run an external Python module.
This will build a class for handling the different program data sets later.

In [ ]:
%run ECHO_modules/DataSet.py
%run ECHO-Cross-Program/utilities.py
%run ECHO-Cross-Program/make_data_sets.py

### 3.  Run this next cell to create the widget to choose whether you want to view facilities by state, county, congressional district or zip code.  Choose the type of region and proceed to the next cell.

In [ ]:
region_type_widget = show_region_type_widget()
state_widget = None

### 4.  Select the state of interest.  If you are interested in a Zip code analysis, this can be skipped.

In [ ]:
state_widget = show_state_widget()

### 5.  Run this cell for all types of regions.  
#### Here is a map of congressional districts: https://www.govtrack.us/congress/members/map

In [ ]:
region_widget = None
if ( region_type_widget.value != 'State' ):
    region_widget = show_pick_region_widget( type=region_type_widget.value,
                                           state_widget=state_widget )

### 6. Run this cell. It will pull facility information from the data  in the  ECHO_EXPORTER table.

In [ ]:
region_selected = None
if ( region_type_widget.value != 'State' ):
    region_selected = region_widget.value
echo_data = get_echo_data( region_type=region_type_widget.value, 
                          region_widget=region_widget, state_widget=state_widget )

## Create an interactive map of all of the facilities in the area that report to EPA
### 7. Once you run this cell, a map should appear. You can zoom in and out, or click on facilities to get their names and the EPA programs that monitor them.

The EPA program acronyms are:
- CAA = Clean Air Act
- CWA = Clean Water Act
- SDWIS = Safe Drinking Water Information System
- RCRA = Resource Conservation and Recovery Act
- TRI = Toxics Release Inventory
- GHG = Greenhouse Gas

The map won't display if there are too many markers, so only the first 400 are shown. 
The facilities shown are selected in order of the ECHO_EXPORTER table's FAC_QTRS_WITH_NC field, which is defined as a "Count of the number of quarters, out of the last twelve quarters, in which the permit or site is considered either in Noncompliance (NC), Significant Noncompliance (SNC) or High Priority violation (HPV) status."

In [ ]:
df_to_map = echo_data
if ( len(df_to_map) > 400 ):
    df_to_map = echo_data[echo_data["FAC_QTRS_WITH_NC"] > 0].sort_values(by="FAC_QTRS_WITH_NC", ascending=False)
    
map_of_facilities = mapper(df_to_map)
map_of_facilities

## In this section you may save the region's data to your computer.
### 8. This next cell allows you to save the data from ECHO_EXPORTER for your chosen region to your computer.  
#### Note: This will save all of the facilities, monitored under any EPA program.  You have the opportunity later to save information on only those facilities monitored under a specific EPA program.
_Note: When you click on [] in the cell, it may continue to show \*. That's to be expected! Check your Downloads folder and confirm that the spreadsheet was succesfully exported. Hit the square button (Interrupt Kernel) at the top of the page. You can now choose to export other data sets from the dropdown..._

In [ ]:
state_name = None
if ( state_widget is not None ):
    state_name = state_widget.value
write_file( df=echo_data, base='ECHO_EXPORTER', type=region_type_widget.value, 
           state=state_name, region=region_selected )

We need to set up some data structures in this next cell that will be used in the remainder of the report.

## Graph program-specific data for all the facilities in the region.
### First, choose the program (Air, Water, or RCRA) and type of data (Violations, Inspections, Enforcement) you want to explore

Running the code below will show you a dropdown you can use to make a selection. **Inspections** or **evaluations** are usually the first phase in the enforcement pipeline, how many (but not all) violations are discovered.  **Violations** happen when a facilty is found to be out of compliance with an environmental law, often by violating the terms of its permit issued under that law.  **Enforcement actions** happen when the regulating agency decides to initiate a "case" against a firm for its violation of an environmental law, either by considering some administrative penalty such as a fine or by taking the violator to court. 

What's available for you to look at here:

Hazardous and Other Waste:
- RCRA_EVALUATIONS = Inspections under RCRA
- RCRA_VIOLATIONS = Violations of RCRA rules
- RCRA_ENFORCEMENTS = Enforcement actions taken by state agencies and the EPA

Here's where you can find definitions of terms used in the columns of RCRA data: 
https://echo.epa.gov/tools/data-downloads/rcrainfo-download-summary 

Air:
- ICIS-AIR_VIOLATION_HISTORY = CAA violations
- ICIS-AIR_FCES_PCES = Both state and federal CAA compliance evaluations
- ICIS-AIR_FORMAL_ACTIONS = CAA formal enforcement actions

Here's where you can find definitions of terms used in the columns of Clean Air Act data: 
https://echo.epa.gov/tools/data-downloads/icis-air-download-summary

Air Emissions (Combined air emissions data for stationary sources from four EPA air programs: National Emissions Inventory (NEI), Greenhouse Gas Reporting Program (GHGRP), Toxic Release Inventory (TRI), and Clean Air Markets (CAMD)):
- Greenhouse Gases via POLL_RPT_COMBINED_EMISSIONS
- Toxic Releases via POLL_RPT_COMBINED_EMISSIONS

Here's where you can find definitions of terms used in the columns of this data: https://echo.epa.gov/tools/data-downloads/air-emissions-download-summary

Water (National Pollutant Discharge Elimination System, or NPDES):
- NPDES_QNCR_HISTORY = CWA Quarterly Non-Compliance History
- NPDES_INSPECTIONS = CWA Inspections
- NPDES_FORMAL_ENFORCEMENT_ACTIONS = CWA Enforcements

Here's where you can find definitions of terms used in the columns of Clean Water Act data: https://echo.epa.gov/tools/data-downloads/icis-npdes-download-summary

Drinking Water (Safe Drinking Water Act, or SDWA):
- SDWA_PUB_WATER_SYSTEMS = Public Water Systems under Safe Drinking Water Act
- SDWA_SITE_VISITS = Safe Drinking Water Site Visits
- SDWA_VIOLATIONS = Safe Drinking Water Violations
- SDWA_SERIOUS_VIOLATORS = Safe Drinking Water Serious Violators
- SDWA_ENFORCEMENTS = Safe Drinking Water Enforcements
- SDWA_RETURN_TO_COMPLIANCE = Safe Drinking Water Return to Compliance

Here's where you can find definitions of terms used in the columns of Clean Water Act data: https://echo.epa.gov/tools/data-downloads/sdwa-download-summary


### 9. This cell creates data sets for facilities in the region and presents a dropdown selection.

In [ ]:
data_sets = make_data_sets()

data_set_widget = show_data_set_widget( data_sets, echo_data )

### 10. Once you've chosen the program and data type you want to focus on...
....run the code below, which will access our database and retrieve the information

In [ ]:
program = data_sets[ data_set_widget.value ]
program_data = get_program_data( program, echo_data )
program_data

### 11.  In this cell you may save the program data for your region to your computer.
This next cell allows you to save the data from the particular program for your chosen region to your computer.

_Note: When you click on [] in the cell, it may continue to show \*. That's to be expected! Check your Downloads folder and confirm that the spreadsheet was succesfully exported. Hit the square button (Interrupt Kernel) at the top of the page. You can now choose to export other data sets from the dropdown..._

In [ ]:
if ( len( program_data ) > 0 ):
    state_name = None
    if ( state_widget is not None ):
        state_name = state_widget.value
    write_file( df=program_data, base=program.name, type=region_type_widget.value, 
           state=state_name, region=region_selected )
else:
    print( "There is no data for this program and region." )

### 12. Let's make a chart out of this!

In [ ]:
state = None
if ( state_widget ):
    state = state_widget.value
    
show_chart( program=program, region=region_selected, data=program_data, state=state )

### 13. Show the facilities in the region in this data set.

In [ ]:
map_of_facilities = mapper(df=program_data, is_echo=False)
map_of_facilities

### If you'd like to look at another dimension of ECHO data for this region...
....return to the dropdown menu above and choose a different program/data type, run the database access code, and then chart and map it!

---

## Choose a specific facility
### 14. In the next few blocks of code, you can focus in on just one of these facilities in the region.
Run the below cell, then choose a facility from the dropdown that appears.

In [ ]:
fac_series = echo_data["FAC_NAME"]
fac_widget = show_fac_widget( fac_series )

### 15. The next cell filters the program-specific IDs to get just records for the selected facility.
Running this cell will create a dropdown list of the program data sets available specifically for this faciliity. Pick one and then run the second cell to view its contents.

In [ ]:
fac_data = echo_data[echo_data["FAC_NAME"] == fac_widget.value]
fac_pgm_widget = show_fac_pgm_widget( fac_data=fac_data, data_sets=data_sets )

### 16. Let's see the data for this facility

In [ ]:
fac_pgm_data = get_program_data( data_sets[ fac_pgm_widget.value ], fac_data )

if ( fac_pgm_data is None or len( fac_pgm_data ) == 0 ):
    print( "There is no {} data for {} ".format( fac_pgm_widget.value, fac_widget.value ))
else:
    print( fac_pgm_data )

### 17. We can make a chart

In [ ]:
if ( fac_pgm_data is None or len( fac_pgm_data ) == 0 ):
    print( "There is no data to chart.")
else:
    show_chart( program=data_sets[ fac_pgm_widget.value ], region=region_selected, data=fac_pgm_data, state=state, fac_name=fac_widget.value )


### 18. This cell saves this facility's data for this program to your computer.
The next cell exports this data to your computer.  You can return to the dropdown to export additional data files.

_Note: When you click on [] in the cell, it may continue to show \*. That's to be expected! Check your Downloads folder and confirm that the spreadsheet was succesfully exported. Hit the square button (Interrupt Kernel) at the top of the page. You can now choose to export other data sets from the dropdown..._

In [ ]:
# fac_pgm_data = add_fac_to_pgm_data( fac_pgm_data, fac_data )
if ( len( fac_pgm_data ) > 0 ):
    state_name = None
    if ( state_widget is not None ):
        state_name = state_widget.value
    write_file( df=fac_pgm_data, base= fac_pgm_widget.value+ '-' + fac_widget.value, 
               type=region_type_widget.value, state=state_name, region=region_selected )
else:
    print( "There is no data for this program and region." )

### Accessing your files
Click on the 'Files' tab in the menu on the left-hand side of the notebook (it looks like a folder). You may have to hit 'Refresh' if you don't see your file. Then, you can click on the ... next to your file and choose "Download". The CSV spreadsheet will download to wherever your browser usually saves files (e.g. Downloads folder)

### If you'd like to look at another dimension of ECHO data for this facility...
....return to the dropdown menu above and choose a different program/data type, then chart it!